In [1]:
def VERBpipeline(x):
    x._.att_li["aux_labels"] = []
    if x._.att_li["family"]=="":
        x._.att_li["family"] = "VERB"
        x._.att_li["subtype"] = "MAIN"
    morphoverb = x.morph.to_dict()
    for i in x.children:
        if i.dep_ == "svp":
            i._.att_li["family"] = "VERB" #+= ["SEPARABLE_VERB_p"]
            i._.att_li["subtype"] = "SVP"
            
            x._.att_li["aux_labels"] += ["SEPARABLE_VERB"]
            x._.att_li["aux_string"] = i.text
        elif i.pos_ == "AUX":
            i._.att_li["family"] = "VERB"
            i._.att_li["subtype"] = "AUX"
        else:
            adder=0
            morphochild = i.morph.to_dict()
            if morphochild.get("Reflex","No")=="Yes":
                x._.att_li["aux_labels"] += ["REFLEXIVE_VERB"]
                i._.att_li["family"] = "VERB"
                i._.att_li["subtype"] = "REFLEXPRON"
            elif ("Number" in morphoverb) and ("Person" in morphoverb):
                if (i.pos_=="PRON") and (morphochild["PronType"]=="Prs"):
                    for j in ["Number","Person"]:
                        adder+=(morphochild[j]==morphoverb[j])
                    if adder == 2:
                        #i._.extra_info += i._.extra_info+["REFLEXIVE_VERB_p"]
                        i._.att_li["family"] = "VERB"
                        i._.att_li["subtype"] = "REFLEXPRON"
                        x._.att_li["aux_labels"] += ["REFLEXIVE_VERB"]
                        
def nonVERBpipeline(x):
    x._.att_li["aux_labels"] = []
    if x.pos_ in ["NOUN", "PRON", "PROPN", "ADJ", "DET"]:
        if x._.att_li["family"] == "":
            x._.att_li["family"] = "NOUN"
            x._.att_li["subtype"] = x.pos_
        morphodict = x.morph.to_dict()
        x._.att_li["aux_labels"] += [morphodict[i].upper() for i in morphodict if i in ["Gender","Case","Number"]]
    elif x.pos_ not in ["X","SPACE","PUNCT"]:
        if x._.att_li["family"] == "":
                x._.att_li["family"] = "OTHERS"
                x._.att_li["subtype"] = x.pos_
                
def computeattli(doc):
    for i in doc:
        if i.pos_ in ["VERB","AUX"]:
            VERBpipeline(i)
        else:
            nonVERBpipeline(i)
        #print(i,i._.att_li)

In [2]:
def formattokens(x, case = True, gender = True, number = True):
    attributes = x._.att_li
    formatted = x.text
    if attributes["family"]=="VERB":
        formatted="<b>{}</b>".format(formatted)
    else:
        if case:
            if "Nom" in attributes["aux_labels"]:
                formatted = '<span class="doubleUnderline">{}</span>'.format(formatted)
            if "Acc" in attributes["aux_labels"]:
                formatted = '<span class="normalUnderline">{}</span>'.format(formatted)
            if "Dat" in attributes["aux_labels"]:
                formatted = '<span class="dashedUnderline">{}</span>'.format(formatted)
            if "Gen" in attributes["aux_labels"]:
                formatted = '<span class="dottedUnderline">{}</span>'.format(formatted)
        if gender:
            if "Masc" in attributes["aux_labels"]:
                formatted ='<span style="color: blue">{}</span>'.format(formatted)
            if "Fem" in attributes["aux_labels"]:
                formatted ='<span style="color: pink">{}</span>'.format(formatted)
            if "Neut" in attributes["aux_labels"]:
                formatted ='<span style="color: orange">{}</span>'.format(formatted)
        if number:
            if "Plur" in attributes["aux_labels"]:
                formatted ='<span class="normalOverline">{}</span>'.format(formatted)
            if "Sing" in attributes["aux_labels"]:
                #formatted ='<span style="color: orange">{}</span>'.format(formatted)
                pass

    x._.html_formatted= formatted
        
def computeformat(doc):
    for i in doc:
        formattokens(i)
        
def printFormattedText(doc):
    initialstring = ""
    html = '<!DOCTYPE html><html lang="en"><head><link href="stylevisualaid.css" rel="stylesheet"></head><body><p style="line-height: 200%">{}</p></body></html>'
    for i in doc:
        if i.is_punct==True:
            initialstring+=i._.html_formatted
        else:
            initialstring+= " "+ i._.html_formatted
    with open("visualaid.html","w") as fp:
        fp.write(html.format(initialstring.replace("\n","</br>")))

In [3]:

def findword(ff,x):
    #ff = pd.read_csv(vocabfile)
    ffx = ff[ff["entries"].apply(lambda y: x in y)].sort_values("nwords")
    if len(ffx)==1:
        return ffx.iloc[0].to_dict()
    ffx = ffx[ffx["nwords"]==ffx["nwords"].min()]
    if len(ffx)==1:
        return ffx.iloc[0].to_dict()
    return {}
def enrichnewvocabulary(newvocdf,ff):

    enrichment = newvocdf["word"].apply(lambda x: findword(ff,x))
    enrichment = enrichment.apply(pd.Series)
    listenrichedwords = enrichment["entries"].to_list()
    return pd.concat([newvocdf,enrichment[["entries","examples","translation"]]],axis=1),listenrichedwords

def newwordchecker(doc,wordlistname,vocabfile,texttitle):

    today = date.today()
    d1 = today.strftime("%Y/%m/%d")
    try:
        wordlist = pd.read_csv(wordlistname, sep ='|')
    except:
        wordlist = pd.DataFrame(columns = ["word","pos","firsttime","lasttime","counter"])
    newwords = []
    for x in doc:
        if x.pos_ not in ["NUM","X","PROPN","SPACE","PART","PUNCT"]:
            rowword = wordlist[wordlist["word"]==x._.att_li["aux_string"]+x.lemma_]
            if len(rowword)==1:
                index = rowword.index[0]
                if x.pos_ not in wordlist.loc[index,"pos"]:
                    wordlist.loc[index,"pos"] += ", "+x.pos_
                wordlist.loc[index,"lasttime"] = today.strftime("%Y/%m/%d")
                wordlist.loc[index,"counter"] += 1
            if len(rowword)==0:
                x._.newword= True
                newwords.append({"word":x._.att_li["aux_string"]+x.lemma_,"pos": x.pos_, "firsttime":d1, "lasttime":d1,"counter":1})
    newvocdf = pd.DataFrame(newwords)
    ff = pd.read_csv(vocabfile, sep = "|")
    report,listenriched = enrichnewvocabulary(newvocdf, ff)
    #render dataframe as html
    html = report.to_html()

    #write html to file
    text_file = open("report_"+texttitle+".html", "w")
    text_file.write(html)
    text_file.close()
    
    pd.concat([wordlist, report]).to_csv(wordlistname, sep ='|', index= False)

    if "counter" not in ff.columns:
        ff["counter"] = 0
        ff["firsttime"] = "never"
        ff["lasttime"] = "never"
        
    ff.loc[ff["entries"].apply(lambda x: x in listenriched), "counter"]+=1
    ff.loc[(ff["entries"].apply(lambda x: x in listenriched))&(ff["firsttime"]=="never"), "firsttime"] = d1
    ff.loc[ff["entries"].apply(lambda x: x in listenriched), "lasttime"]=d1
    ff.to_csv(vocabfile, sep ='|', index= False)




In [4]:


        
def extractnewword(sentence):
    tokens = pd.Series(sentence)
    df = tokens.apply(lambda x: {"token":x,"newword":x._.newword,"qc":x._.questioncategory}).apply(pd.Series)
    filtered = df[df["newword"]&(df["qc"]!=0)]
    if len(filtered)==0:
        return {"question":"no_question"}
    chosenword = filtered.sample(frac=1).iloc[0]
    return {"question": sentence, "word": chosenword["token"],"qc":chosenword["qc"]}

def other3(word, qc,qcdf):
    if qc>0:
        inddel = qcdf[(qcdf["qc"]==qc)&(qcdf["word"]==word.text)].index
        candidates = qcdf[(qcdf["qc"]==qc)].drop(inddel)
    if qc=="VERB":
        inddel = verbs[verbs["word"]==word.text].index
        candidates = verbs.drop(inddel)
    return [word.text]+candidates.sample(n = 3)["word"].to_list()

def createoption(option, number,correct=False):
    if correct:
        output = """
        <input class="answer" type="radio" name="q{}" value="1">
        <label id="correctString{}">{}</label>
        """.format(number, number, option)
    else:
        output = """
        <input class="answer" type="radio" name="q{}" value="0">
        <label>{}</label>
        """.format(number, option)
    return output

def answer(n):
    output = """
            if (answerScore('[cor]') === 0) {
                document.getElementById('correctAnswer[cor]').innerHTML = correctAnswer('correctString[cor]', [cor]);
            }
    """.replace("[cor]",str(n))
    return output

def createquestion(question, options, number):
    op1 = createoption(options[0],number,correct=True)
    op2 = createoption(options[1],number)
    op3 = createoption(options[2],number)
    op4 = createoption(options[3],number)
    formatedoptions = [op1,op2,op3,op4]
    random.shuffle(formatedoptions)
    output = """
<tr>
<div>
<p class="question">{}. {}</p>
<ul>
{}
<br>
{}
<br>
{}
<br>
{}
</ul>
</div>
</tr>""".format(number, question,formatedoptions[0],formatedoptions[1],formatedoptions[2],formatedoptions[3])
    return output



def quizpipeline(doc,texttitle):
    sents = pd.Series(doc.sents)
    qcdf = pd.Series(doc).apply(lambda x: {"word":x.text,"qc":x._.questioncategory}).apply(pd.Series).drop_duplicates()
    questionaire = sents.apply(extractnewword).apply(pd.Series)
    questionaire=questionaire[questionaire["question"]!="no_question"]
    questionaire["options"]=questionaire.apply(lambda x: other3(x["word"],x["qc"],qcdf),axis=1)
    questionaire["question_text"] = questionaire.apply(lambda x:x["question"].text.replace(x["options"][0],"___________"),axis=1)

    all_questions = {}
    for i in range(len(questionaire)):
        all_questions[questionaire.iloc[i]["question_text"].replace("\n","")]=questionaire.iloc[i]["options"]

    ind=0
    formatedquestions = ""
    for i in all_questions:
        ind+=1
        formatedquestions+=createquestion(i,all_questions[i],ind)

    start = """
    <!DOCTYPE html>

    <html lang="en">
    <head>
    <link href="elcss.css" rel="stylesheet">
    </head>
    <body>
      <h2 class="quizHeader">Take a Quiz!</h2>

        <table style="width:583px">
    """

    end="""
        </table>
    <br/>
      <div class="submitter">
              <input class="quizSubmit" id="submitButton" onClick="submitQuiz()"
              type="submit" value="Submit" />
        </div>

    <!--show only wrong answers on submit-->
    """
    correctanswers = ""
    for i in range(1,ind+1):
        correctanswers+= '<div class="quizAnswers" id="correctAnswer{}"></div>'.format(i)

    end2 = """
    <!--show score upon submit-->
        <div>
            <h2 class="quizScore" id="userScore"></h2>
        </div>
        </div>
        <script src="eljstexttitle.js"></script>
    </body>

    </html>

    """.replace("texttitle",texttitle)

    finalhtml=start+formatedquestions+end+correctanswers+end2


    nrq=ind
    part1="""
    // scripts here:

        function submitQuiz() {
            console.log('submitted');

        // get each answer score
            function answerScore (qName) {
                var radiosNo = document.getElementsByName(qName);

                for (var i = 0, length = radiosNo.length; i < length; i++) {
                    if (radiosNo[i].checked) {
                // do something with radiosNo
                        var answerValue = Number(radiosNo[i].value);
                    }
                }
                // change NaNs to zero
                if (isNaN(answerValue)) {
                    answerValue = 0;
                }
                return answerValue;
            }

        // calc score with answerScore function
    """
    aux = []
    for i in range(1,nrq+1):
        aux.append("answerScore('q{}')".format(i))
    part2="		var calcScore = ("+ " + ".join(aux)+");"
    part3="""
            console.log("CalcScore: " + calcScore); // it works!

        // function to return correct answer string
            function correctAnswer (correctStringNo, qNumber) {
                console.log("qNumber: " + qNumber);  // logs 1,2,3,4 after called below
                return ("The correct answer for question #" + qNumber + ": &nbsp;<strong>" +
                    (document.getElementById(correctStringNo).innerHTML) + "</strong>");
                }

        // print correct answers only if wrong (calls correctAnswer function)
    """
    part4=""

    for i in range(1,nrq+1):
        part4+=answer(i)

    part5="""
        // calculate "possible score" integer
            var questionCountArray = document.getElementsByClassName('question');

            var questionCounter = 0;
            for (var i = 0, length = questionCountArray.length; i < length; i++) {
                questionCounter++;
            }

        // show score as "score/possible score"
            var showScore = "Your Score: " + calcScore +"/" + questionCounter;
        // if 4/4, "perfect score!"
            if (calcScore === questionCounter) {
                showScore = showScore + "&nbsp; <strong>Perfect Score!</strong>"
            };
            document.getElementById('userScore').innerHTML = showScore;
        }

    $(document).ready(function() {

        $('#submitButton').click(function() {
            $(this).addClass('hide');
        });

    });
    """

    finaljss=part1+part2+part3+part4+part5

    fw = open("elhtml"+texttitle+".html", "w")
    fw.write(finalhtml)
    fw.close()

    f = open("eljs"+texttitle+".js", "w")
    f.write(finaljss)
    f.close()


def questionClassifier(x):
    subtype = x._.att_li["subtype"]
    if subtype in ["REFLEXPRON", "PRON"]:
        x._.questioncategory = 1
    elif subtype in ["MAIN","AUX"]:
        x._.questioncategory = 2
    elif subtype =="NOUN":
        x._.questioncategory = 3
    elif subtype =="DET":
        x._.questioncategory = 4
    elif subtype == "ADV":
        x._.questioncategory = 5
    elif subtype in ["ADP","SVP"]:
        x._.questioncategory = 6
    elif subtype == "ADJ":
        x._.questioncategory = 7
    elif subtype in ["CCONJ", "SCONJ", "PART"]:
        x._.questioncategory = 8
    else:
        x._.questioncategory = 0
        
def computeQuestions(doc):
    for i in doc:
        questionClassifier(i)
    questionCount = pd.Series(doc).apply(lambda x: x._.questioncategory).value_counts().to_dict()
    delete = [i for i in questionCount if questionCount[i]<4]
    for i in doc:
        if i._.questioncategory in delete:
            i._.questioncategory = 0

In [5]:
import pandas as pd
import spacy
from spacy.tokens import Token
import random
from datetime import date

Token.set_extension('att_li', default={"family":"","subtype":"","aux_labels":[],"aux_string":""},force=True)
#Token.set_extension('svp', default="",force=True)
Token.set_extension("html_formatted", default="",force=True)
Token.set_extension("questioncategory",default="",force=True)
Token.set_extension("newword",default=False,force=True)
nlp = spacy.load("de_core_news_lg")


In [6]:
title1= "title1"
text = """
Welche Eigenschaften haben Menschen, die sich durch innovative Ideen auszeichnen? Was haben sie gemeinsam? Ist es möglich, sie unter Tausenden zu erkennen? Eine sechsjährige Studie versuchte, diese Fragen zu beantworten.
In dem Bewusstsein, dass Innovationsfähigkeit eine wesentliche Eigenschaft für den Erfolg und das Überleben eines Unternehmens ist, fragen sich Top-Manager/-innen oft, wie sie innovative Mitarbeitende finden sowie selbst innovativer werden können. 
Drei Universitätsprofessoren und Forscher haben deshalb sechs Jahre lang innovative Unternehmer/-innen und Manager/-innen beobachtet und interviewt, um zu verstehen, was diese voneinander unterscheidet und den einen zum Steve Jobs und die andere zur Besitzerin eines Franchise-Geschäfts macht. Diese Studie führte zur Entdeckung von fünf essenziellen Fähigkeiten, welche diejenigen charakterisieren, die einhellig als Innovatoren bzw. Innovatorinnen angesehen werden und nach Ansicht der Wissenschaftler das besitzen, was sie als "DNA des Innovators" bezeichnen:
1. Die Fähigkeit, Assoziationen zu bilden bzw. Elemente (Probleme, Ideen, Fragen, ...) zu verknüpfen, die auf den ersten Blick keinen direkten Bezug zueinander haben: Dieses Phänomen wurde von dem Unternehmer Frans Johansson als "Medici-Effekt" beschrieben, welcher sich auf die kreative Explosion in Florenz bezieht, als die Medici-Familie Menschen aus verschiedenen Disziplinen (Dichtung, Bildhauerei, Architektur, Philosophie, Malerei,...) zusammenbrachte und durch die Interaktion zwischen verschiedenen Wissens- und Tätigkeitsbereichen eine Periode außergewöhnlicher Kreativität herbeiführte, die man heute als Renaissance kennt. Das Gehirn assoziiert Ideen basierend auf den Erfahrungen, die gemacht wurden. Je vielfältiger diese Erfahrungen sind, desto mehr ist das Gehirn in der Lage, nicht offensichtliche Assoziationen zu erzeugen.
"""
title2 = "title2"
text2="""
2. Die Fähigkeit, Fragen zu stellen: Wie jemand einmal sagte: "Das Wichtigste ist nicht, die richtigen Antworten zu finden, sondern die richtigen Fragen zu stellen". Während Unternehmer/-innen sich fragen, wie sie bestehende Prozesse verbessern können, denken Innovatoren und Innovatorinnen darüber nach, wie sie den Status quo und die traditionelle Denkweise in Frage stellen können und fragen sich selbst: "Wenn wir XY tun würden, was würde dann passieren"?
3. Die Fähigkeit zu beobachten: Innovatoren und Innovatorinnen beobachten sorgfältig die Welt um sich herum, egal ob es sich um potenzielle Kunden, Lieferanten, konkurrierende Unternehmen oder normale Menschen handelt, die arbeiten und ihren täglichen Aktivitäten nachgehen. Dadurch erhalten sie Einblicke und folglich Ideen, wie bestimmte Dinge auf eine neue Art und Weise erledigt werden können.  
4. Die Fähigkeit zu experimentieren: Genau wie Wissenschaftler und Wissenschaftlerinnen testen auch Innovatoren und Innovatorinnen aktiv ihre Ideen, erstellen Prototypen und initiieren Pilotprojekte. Das Leben und Arbeiten im Ausland ist beispielsweise eine nützliche Erfahrung, um innovative Ideen zu stimulieren: Es scheint, dass, je mehr Länder eine Person besucht hat, desto wahrscheinlicher ist es, dass sie diese Erfahrungen nutzt, um innovative Dienstleistungen oder Produkte zu erschaffen. Tatsächlich zeigen Untersuchungen, dass Unternehmen, die von CEOs geführt werden, welche vor deren Ernennung mindestens eine internationale Erfahrung gesammelt haben, finanziell besser abschneiden als Unternehmen, die von CEOs ohne diese Erfahrung geführt werden. 
5. Networking: Innovative Unternehmer/-innen treffen und sprechen mit Menschen aus verschiedenen Berufsfeldern und mit unterschiedlichen Sichtweisen, um ihr Wissen zu erweitern und die Dinge aus einer anderen Perspektive zu sehen.
Die gute Nachricht ist, dass wir, auch wenn wir nicht mit der richtigen DNA geboren wurden, trotzdem zu Innovator/-innen werden können, indem wir die fünf oben beschriebenen Fähigkeiten regelmäßig trainieren, sodass die entsprechenden Verhaltensweisen mit der Zeit automatisiert werden. Kurz gesagt: Innovatoren und Innovatorinnen werden nicht als solche geboren, sondern zu diesen gemacht. 
Stimmst du den Forschungsergebnissen zu, dass Innovator/-innen nicht geboren, sondern gemacht werden? Sind deiner Meinung nach noch andere Fähigkeiten erforderlich, um innovative Ideen entwickeln zu können?
"""

In [7]:
doc = nlp(text)
assert doc.has_annotation("SENT_START")
computeattli(doc)
computeformat(doc)
printFormattedText(doc)
newwordchecker(doc,"pablohistory.csv","vocabulary.csv",title1)


In [8]:
computeQuestions(doc)
quizpipeline(doc,title1)

In [9]:
doc = nlp(text2)
assert doc.has_annotation("SENT_START")
computeattli(doc)
computeformat(doc)
printFormattedText(doc)
newwordchecker(doc,"pablohistory.csv","vocabulary.csv",title2)
computeQuestions(doc)
quizpipeline(doc,title1)

In [16]:
doc[5]._.


2. Die Fähigkeit, Fragen zu stellen: Wie jemand einmal sagte: "Das Wichtigste ist nicht, die richtigen Antworten zu finden, sondern die richtigen Fragen zu stellen". Während Unternehmer/-innen sich fragen, wie sie bestehende Prozesse verbessern können, denken Innovatoren und Innovatorinnen darüber nach, wie sie den Status quo und die traditionelle Denkweise in Frage stellen können und fragen sich selbst: "Wenn wir XY tun würden, was würde dann passieren"?
3. Die Fähigkeit zu beobachten: Innovatoren und Innovatorinnen beobachten sorgfältig die Welt um sich herum, egal ob es sich um potenzielle Kunden, Lieferanten, konkurrierende Unternehmen oder normale Menschen handelt, die arbeiten und ihren täglichen Aktivitäten nachgehen. Dadurch erhalten sie Einblicke und folglich Ideen, wie bestimmte Dinge auf eine neue Art und Weise erledigt werden können.  
4. Die Fähigkeit zu experimentieren: Genau wie Wissenschaftler und Wissenschaftlerinnen testen auch Innovatoren und Innovatorinnen aktiv ih

In [11]:
1+2+3

6

In [15]:
doc[5].morph

Case=Acc|Gender=Fem|Number=Plur

In [ ]:
see = pd.read_csv("vocabulary.csv",sep="|")

In [ ]:
see[see["counter"]==1]

In [ ]:
word = input1["word"].iloc[0]

In [ ]:
input2.loc[word in input2["entries"]]

In [ ]:
findword(input2,input1["word"].iloc[0])

In [ ]:
input2

In [ ]:
input1

In [ ]:
doc = nlp(text)

In [ ]:
A = pd.DataFrame({"a":[1,2,3],"b":[4,5,6],"c":[7,8,9]})

In [ ]:
pd.DataFrame({"a":[1,2],"b":[4,5]})

In [ ]:
pd.concat({})

In [ ]:


# dd/mm/YY
d1 = 


In [ ]:
def enrichnewvocabulary(newvocdf,vocab, doc):
    #prendere la frase di cui forma parte nel futuro/buscar en google
    

In [ ]:
def createquestions(,)

In [ ]:
pd.concat([pd.Series([1,2,3]),pd.Series([2,3,4])])

In [ ]:
AUX[0].apply(lambda x: x.pos_).value_counts()

In [ ]:
base = pd.Series(doc)
AUX = pd.concat([base,base.apply(lambda x:x._.att_li).apply(pd.Series)],axis=1)


In [ ]:
AUX["subtype"].value_counts()

In [ ]:
set(AUX["aux_labels"].sum())

In [ ]:
def questiongroup(x):
    

In [ ]:
computeattli(doc)

In [ ]:

nonVERBpipeline(doc[9])

In [ ]:
doc[30]._.att_li

In [ ]:
#3 aspects NOUN, PRON, PROPN, ADJ, DET -> noun
# ADV, CCONJ, SCONJ, ADP (sz VERB) -> fixed elements
# AUX, VERB, ADP (cn VERB) -> verb

In [ ]:
def findUV(x):
    dictmorph = x.morph.to_dict()
    for i in dictmorph:
        if i in ["Case","Gender","Number"]:
            i_.extra_info += dictmorph[i].upper()
        if i.dep_ == "svp":
            i._.extra_info += ["SEPARABLE_VERB_p"]
            x._.extra_info += ["SEPARABLE_VERB"]
            x._.svp = i.text

def compute_att_li(token):
    

In [ ]:
AUX["POS"].value_counts()

In [ ]:
#VISUAL AID
#NOUN e PROPN: Colorarli in base al numero, caso e genere
#VERB, AUX, prep de separabili: grassetto

#Vocabolario - significato
#ADJ, ...

#analisi sintatico

#storico



#BASIC:


# Nouns

In [ ]:
AUX["POS"].value_counts()

In [ ]:
AUX[AUX["POS"]=="PART"]

In [ ]:
AUX["POS"].value_counts()

In [ ]:
AUX[AUX["POS"]=="NOUN"]["token"].apply(lambda x: x.morph.to_dict())#.apply(pd.Series)

In [ ]:
nouns = pd.concat([AUX[AUX["POS"]=="NOUN"],AUX[AUX["POS"]=="NOUN"]["morph"].apply(lambda x: x.to_dict()).apply(pd.Series)],axis=1)

# Verbs

In [ ]:
verbs = pd.concat([AUX[AUX["POS"]=="VERB"],AUX[AUX["POS"]=="VERB"]["morph"].apply(lambda x: x.to_dict()).apply(pd.Series)],axis=1)
verbs["token"].apply(findUV)
verbs["token"].apply(findAUX)
verbs["token"].apply(check_reflexive_verb)

In [ ]:
verbs

In [ ]:
verbtok = nlp("Ich ziehe mich an, kämme mich und putze mir die Zähne.")[5]


In [ ]:
nlp("Ich ziehe mich an, kämme mich und putze mir die Zähne.")[2].dep_

In [ ]:
nlp("Ich ziehe mich an, kämme mich und putze mir die Zähne.")[2].pos_

In [ ]:
verbtok.morph

In [ ]:
list(verbtok.children)[0].pos_

In [ ]:
list(verbtok.children)[0].morph.to_dict()

In [ ]:
nlp("sich freuen")[0].morph.to_dict()

In [ ]:
adder

In [ ]:
nlp("Ich ziehe mich an, kämme mich und putze mir die Zähne.")[6].pos_

In [ ]:
spacy.explain('oa')

In [ ]:
!!!!! verbs reflexivos jodeeeeeeeeeeeerrrrrrrrrrr

In [ ]:
verb

In [ ]:
AUX[AUX["POS"]=="PRON"]

In [ ]:
AUX["POS"].value_counts()

In [ ]:
findAUX

In [ ]:
verbs[verbs["aux"].isna()==False]

In [ ]:
nlp("Ich habe alles gemacht")[1].pos_

In [ ]:
spacy.explain("svp")

In [ ]:
verbs

In [ ]:
list(verbs["token"].iloc[9].children)

In [ ]:
morpho.to_dict()